제목 : Naive Bayes Classifier 실습  
학습일 : 2022-07-05  
작성자 : [송유럽(Euro)](https://www.euroverse.dev/)  
관련 링크 : [딥러닝을 이용한 자연어처리 입문](https://wikidocs.net/22892)

--- 설명 ---  
나이브 베이즈 분류기를 공부하다가 '뉴스그룹 데이터 분류하기'를 실습한 코드입니다.


--- 더 공부할 내용 ---  
CountVectorizer 내용 (BoW 또는 DTM 챕터 참고)  
TfidVectorizer 내용 (DTM대신 TF-IDF를 사용했을 때 성능개선되는 이유 등)  
저자가 마지막에 한 말 (여기서는 하지 않았지만, 잠재 의미 분석 챕터에서 진행했던 전처리를 모두 진행하고 다시 나이브 베이즈 분류기를 돌려보세요. 80% 이상의 정확도를 얻을 수 있습니다.)


In [1]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
print(len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

11314 11314 20 11314


In [17]:
print("첫번째 샘플의 카테고리 인덱스: "+str(newsdata.target[0]))
print("7번 인덱스에 해당하는 카데고리: "+str(newsdata.target_names[7]))
print("-------첫번째 샘플의 내용-------", newsdata.data[0], sep='\n')

첫번째 샘플의 카테고리 인덱스: 7
7번 인덱스에 해당하는 카데고리: rec.autos
-------첫번째 샘플의 내용-------
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







아래 코드는 newsdata.data를 전처리하는 코드.  
나이브 베이즈 분류를 위해 백오브워즈를 이용해 워드 임베딩을 한다.  
(백오브워즈는 순서를 고려하지 않고 빈도에 집중해서 토큰화함.)

참고로, 자동으로 BoW로 만드는 CountVectorizer를 이용함.  
(BoW 또는 DTM 챕터를 참고하자.)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [20]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


위의 11314는 훈련용 샘플의 개수,  
130107은 전체 훈련 데이터에 등장한 단어의 수를 의미한다.

위 DTM을 그대로 사용할 수도 있지만,  
DTM 행렬 대신 TF-IDF 행렬을 사용하면 성능 개선을 얻을 수 있으므로  
사이킷런의 TfidVectorizer 클래스를 이용합니다.

In [21]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


이제 본격적으로 나이브 베이즈 분류를 수행해보자!

In [22]:
mod = MultinomialNB()
mod.fit(tfidfv, newsdata.target)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514
